In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import pandas as pd
import time


In [2]:
#Load the CIFAR10 data as tensors

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:
# Plot the images with labels
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

dataiter = iter(trainloader)
images, labels = dataiter.next()

# get some random training images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

<Figure size 640x480 with 1 Axes>

  car  deer   dog   car


### Create dataframes to store results


In [4]:
import pandas as pd
performance = pd.DataFrame(columns=['performance_measure','relu','elu','selu','tanh','logsigmoid','sigmoid'])


In [5]:
performance['performance_measure']=('Total accuracy','Training time','Accuracy of plane',
                                    'Accuracy of car','Accuracy of bird',
                                   'Accuracy of cat','Accuracy of  deer','Accuracy of   dog','Accuracy of  frog',
                                   'Accuracy of horse','Accuracy of  ship','Accuracy of truck')

### RELU

In [15]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [16]:
#Specify learning rate and momentum for the Neural Network
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [17]:
start = time. time()
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
end = time. time()
performance.loc[performance['performance_measure']=='Training time','relu']=round(end - start,5)
print('Finished Training')

[1,  2000] loss: 2.201
[1,  4000] loss: 1.875
[1,  6000] loss: 1.681
[1,  8000] loss: 1.606
[1, 10000] loss: 1.517
[1, 12000] loss: 1.480
[2,  2000] loss: 1.406
[2,  4000] loss: 1.383
[2,  6000] loss: 1.348
[2,  8000] loss: 1.333
[2, 10000] loss: 1.302
[2, 12000] loss: 1.288
Finished Training


In [18]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

performance.loc[performance['performance_measure']=='Total accuracy','relu']=100 * correct / total

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 55 %


In [19]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    performance['relu'][i+2]= round(100 * class_correct[i] / class_total[i],1)
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane : 70 %
Accuracy of   car : 61 %
Accuracy of  bird : 45 %
Accuracy of   cat : 55 %
Accuracy of  deer : 33 %
Accuracy of   dog : 30 %
Accuracy of  frog : 70 %
Accuracy of horse : 60 %
Accuracy of  ship : 54 %
Accuracy of truck : 73 %


#### ELU

In [27]:
## TRAINING NN with ELU
start = time. time()
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.elu(self.conv1(x)))
        x = self.pool(F.elu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.elu(self.fc1(x))
        x = F.elu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

#Specify learning rate and momentum for the Neural Network
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

start = time. time()
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
end = time. time()
performance.loc[performance['performance_measure']=='Training time','elu']=round(end - start,5)
print('Finished Training')

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

performance.loc[performance['performance_measure']=='Total accuracy','elu']=100 * correct / total

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    performance['elu'][i+2]= round(100 * class_correct[i] / class_total[i],1)
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))


[1,  2000] loss: 2.000
[1,  4000] loss: 1.656
[1,  6000] loss: 1.543
[1,  8000] loss: 1.487
[1, 10000] loss: 1.408
[1, 12000] loss: 1.364
[2,  2000] loss: 1.305
[2,  4000] loss: 1.297
[2,  6000] loss: 1.252
[2,  8000] loss: 1.230
[2, 10000] loss: 1.208
[2, 12000] loss: 1.185
Finished Training
Accuracy of the network on the 10000 test images: 58 %
Accuracy of plane : 72 %
Accuracy of   car : 67 %
Accuracy of  bird : 49 %
Accuracy of   cat : 20 %
Accuracy of  deer : 59 %
Accuracy of   dog : 59 %
Accuracy of  frog : 65 %
Accuracy of horse : 63 %
Accuracy of  ship : 68 %
Accuracy of truck : 58 %


In [ ]:
performance

In [28]:
## TRAINING NN with SELU
start = time. time()
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.selu(self.conv1(x)))
        x = self.pool(F.selu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.selu(self.fc1(x))
        x = F.selu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

#Specify learning rate and momentum for the Neural Network
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

start = time. time()
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
end = time. time()
performance.loc[performance['performance_measure']=='Training time','selu']=round(end - start,5)
print('Finished Training')

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

performance.loc[performance['performance_measure']=='Total accuracy','selu']=100 * correct / total

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    performance['selu'][i+2]= round(100 * class_correct[i] / class_total[i],1)
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))


[1,  2000] loss: 1.850
[1,  4000] loss: 1.628
[1,  6000] loss: 1.523
[1,  8000] loss: 1.456
[1, 10000] loss: 1.412
[1, 12000] loss: 1.342
[2,  2000] loss: 1.293
[2,  4000] loss: 1.278
[2,  6000] loss: 1.275
[2,  8000] loss: 1.261
[2, 10000] loss: 1.241
[2, 12000] loss: 1.224
Finished Training
Accuracy of the network on the 10000 test images: 56 %
Accuracy of plane : 57 %
Accuracy of   car : 77 %
Accuracy of  bird : 62 %
Accuracy of   cat : 25 %
Accuracy of  deer : 40 %
Accuracy of   dog : 44 %
Accuracy of  frog : 65 %
Accuracy of horse : 64 %
Accuracy of  ship : 62 %
Accuracy of truck : 58 %


In [29]:
## TRAINING NN with tanh
start = time. time()
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.tanh(self.conv1(x)))
        x = self.pool(F.tanh(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

#Specify learning rate and momentum for the Neural Network
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

start = time. time()
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
end = time. time()
performance.loc[performance['performance_measure']=='Training time','tanh']=round(end - start,5)
print('Finished Training')

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

performance.loc[performance['performance_measure']=='Total accuracy','tanh']=100 * correct / total

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    performance['tanh'][i+2]= round(100 * class_correct[i] / class_total[i],1)
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/functional.py:1320: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


[1,  2000] loss: 2.014
[1,  4000] loss: 1.706
[1,  6000] loss: 1.616
[1,  8000] loss: 1.550
[1, 10000] loss: 1.505
[1, 12000] loss: 1.443
[2,  2000] loss: 1.396
[2,  4000] loss: 1.357
[2,  6000] loss: 1.325
[2,  8000] loss: 1.313
[2, 10000] loss: 1.285
[2, 12000] loss: 1.270
Finished Training
Accuracy of the network on the 10000 test images: 55 %
Accuracy of plane : 71 %
Accuracy of   car : 68 %
Accuracy of  bird : 25 %
Accuracy of   cat : 23 %
Accuracy of  deer : 47 %
Accuracy of   dog : 47 %
Accuracy of  frog : 69 %
Accuracy of horse : 67 %
Accuracy of  ship : 68 %
Accuracy of truck : 62 %


In [34]:
## TRAINING NN with logsigmoid
start = time. time()
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.logsigmoid(self.conv1(x)))
        x = self.pool(F.logsigmoid(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.logsigmoid(self.fc1(x))
        x = F.logsigmoid(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

#Specify learning rate and momentum for the Neural Network
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

start = time. time()
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
end = time. time()
performance.loc[performance['performance_measure']=='Training time','logsigmoid']=round(end - start,5)
print('Finished Training')

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

performance.loc[performance['performance_measure']=='Total accuracy','logsigmoid']=100 * correct / total

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    performance['logsigmoid'][i+2]= round(100 * class_correct[i] / class_total[i],1)
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))


[1,  2000] loss: 2.318
[1,  4000] loss: 2.234
[1,  6000] loss: 2.041
[1,  8000] loss: 2.002
[1, 10000] loss: 1.966
[1, 12000] loss: 1.922
[2,  2000] loss: 1.824
[2,  4000] loss: 1.785
[2,  6000] loss: 1.731
[2,  8000] loss: 1.696
[2, 10000] loss: 1.656
[2, 12000] loss: 1.659
Finished Training
Accuracy of the network on the 10000 test images: 41 %


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Accuracy of plane : 30 %
Accuracy of   car : 62 %
Accuracy of  bird : 33 %
Accuracy of   cat : 23 %
Accuracy of  deer : 24 %
Accuracy of   dog : 36 %
Accuracy of  frog : 57 %
Accuracy of horse : 45 %
Accuracy of  ship : 76 %
Accuracy of truck : 29 %


In [37]:
performance

,performance_measure,relu,elu,selu,prelu,tanh,logsigmoid,sigmoid
0,Total accuracy,55.45,58.6,56.02,41.98000,55.08,41.98000,NaN
1,Training time,194.997,202.082,221.959,291.23494,203.997,291.23494,NaN
2,Accuracy of plane,70,72.9,57.8,30.90000,71,30.90000,NaN
3,Accuracy of car,61.6,67.2,77.1,62.20000,68.8,62.20000,NaN
4,Accuracy of bird,45,49.7,62.8,33.40000,25,33.40000,NaN
5,Accuracy of cat,55.1,20.9,25.4,23.00000,23.1,23.00000,NaN
6,Accuracy of deer,33,59.5,40.6,24.10000,47.6,24.10000,NaN
7,Accuracy of dog,30.1,59.5,44.7,36.70000,47.2,36.70000,NaN
8,Accuracy of frog,70.8,65.9,65.4,57.60000,69.4,57.60000,NaN
9,Accuracy of horse,60.6,63.8,64.9,45.70000,67.5,45.70000,NaN


In [38]:
## TRAINING NN with sigmoid
start = time. time()
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.sigmoid(self.conv1(x)))
        x = self.pool(F.sigmoid(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.sigmoid(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

#Specify learning rate and momentum for the Neural Network
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

start = time. time()
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
end = time. time()
performance.loc[performance['performance_measure']=='Training time','sigmoid']=round(end - start,5)
print('Finished Training')

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

performance.loc[performance['performance_measure']=='Total accuracy','sigmoid']=100 * correct / total

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    performance['sigmoid'][i+2]= round(100 * class_correct[i] / class_total[i],1)
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[1,  2000] loss: 2.316
[1,  4000] loss: 2.313
[1,  6000] loss: 2.314
[1,  8000] loss: 2.313
[1, 10000] loss: 2.312
[1, 12000] loss: 2.311
[2,  2000] loss: 2.310
[2,  4000] loss: 2.308
[2,  6000] loss: 2.309
[2,  8000] loss: 2.309
[2, 10000] loss: 2.307
[2, 12000] loss: 2.308
Finished Training
Accuracy of the network on the 10000 test images: 10 %


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Accuracy of plane :  0 %
Accuracy of   car :  0 %
Accuracy of  bird :  0 %
Accuracy of   cat :  0 %
Accuracy of  deer :  0 %
Accuracy of   dog : 100 %
Accuracy of  frog :  0 %
Accuracy of horse :  0 %
Accuracy of  ship :  0 %
Accuracy of truck :  0 %


In [48]:
performance

,performance_measure,relu,elu,selu,tanh,logsigmoid,sigmoid
0,Total accuracy,55.45,58.6,56.02,55.08,41.98000,10
1,Training time,194.997,202.082,221.959,203.997,291.23494,178.934
2,Accuracy of plane,70,72.9,57.8,71,30.90000,0
3,Accuracy of car,61.6,67.2,77.1,68.8,62.20000,0
4,Accuracy of bird,45,49.7,62.8,25,33.40000,0
5,Accuracy of cat,55.1,20.9,25.4,23.1,23.00000,0
6,Accuracy of deer,33,59.5,40.6,47.6,24.10000,0
7,Accuracy of dog,30.1,59.5,44.7,47.2,36.70000,100
8,Accuracy of frog,70.8,65.9,65.4,69.4,57.60000,0
9,Accuracy of horse,60.6,63.8,64.9,67.5,45.70000,0
